In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service

import time
import pandas as pd
import requests
import numpy as np
import csv

pd.set_option('display.max_colwidth', 20)

In [34]:
# read csv
df = pd.read_csv("../data/cleaned/Posts_Industry.csv")
df.head()

,Unnamed: 0,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack
0,0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set()
1,1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set()
2,2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '..."
3,3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...
4,4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...


In [35]:
# Convert string to list
def str_list(text):
    if text == 'NaN':
        return np.nan
    
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace("'", '')
    mid_working = text.strip().split(',')
    to_be_returned = mid_working[0]
    return to_be_returned

In [36]:
# extract company and industry 
com_industry = df[['company_name', 'industry']]
com_industry.drop_duplicates(inplace=True)
com_industry['industry'] = com_industry['industry'].apply(lambda x: str_list(str(x)))
com_industry

<ipython-input-36-e762d862ddd2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_industry.drop_duplicates(inplace=True)
<ipython-input-36-e762d862ddd2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  com_industry['industry'] = com_industry['industry'].apply(lambda x: str_list(str(x)))


,company_name,industry
0,Citi,Banking/Accounti...
1,Amazon,Internet/E-commerce
2,Ascend Com Pte. ...,Corporate office
3,Applied Material...,Semi conductor s...
7,Recruit Expert P...,Employment agenc...
...,...,...
11417,Global Fashion G...,Training centre ...
11431,Epsilon,Management consu...
11443,2C2P,Software company...
11445,Quilt.AI,None


## Check & clean data

In [6]:
# check for number of unique companies
len(com_industry['company_name'].unique())

2235

In [7]:
df_combine = com_industry.groupby('company_name').agg(sum).reset_index()
df_combine

,company_name,industry
0,Hitachi Vantara,[Database manage...
1,Novade Solutions,[Software compan...
2,Partners Group,[Financial insti...
3,1exchange,[None]
4,21 United Holdin...,[None]
...,...,...
2230,iZeeM,[Business to bus...
2231,iZeno Pte Ltd,[None]
2232,page personnel,[IT/Software Dev...
2233,the LEGO Group,[Toy company]


In [8]:
def remove_irrelevant(text_list):
    
    cleaned_list = []
    for i in text_list:
        if i != 'nan' and i != 'Other':
            cleaned_list.append(i)
            
    if len(cleaned_list) == 0:
        return None
            
    return cleaned_list

In [9]:
df_combine['industry'] = df_combine['industry'].apply(lambda x: remove_irrelevant(x))
df_combine

,company_name,industry
0,Hitachi Vantara,[Database manage...
1,Novade Solutions,[Software compan...
2,Partners Group,[Financial insti...
3,1exchange,[None]
4,21 United Holdin...,[None]
...,...,...
2230,iZeeM,[Business to bus...
2231,iZeno Pte Ltd,[None]
2232,page personnel,[IT/Software Dev...
2233,the LEGO Group,[Toy company]


In [10]:
scrape_df = df_combine[df_combine['industry'].isnull()]
scrape_df

,company_name,industry
970,Infosight Softwa...,None


## Scrape Industry

In [20]:
url = "https://www.google.com.sg/search?q=google.com&source=hp&ei=GJtBYo-vNPKKr7wPq72NmAI&iflsig=AHkkrS4AAAAAYkGpKKctBnYlyKka_pAywfzdvHQ8ymzH&ved=0ahUKEwiPm-CG2ej2AhVyxYsBHateAyMQ4dUDCAk&uact=5&oq=google.com&gs_lcp=Cgdnd3Mtd2l6EAMyCwgAEIAEELEDEIMBMgsIABCABBCxAxCDATILCAAQgAQQsQMQgwEyCwgAEIAEELEDEIMBMgUIABCABDIFCAAQgAQyBQgAEIAEMgUIABCABDIFCAAQgAQyBQgAEIAEOhEILhCABBCxAxCDARDHARDRAzoOCC4QgAQQsQMQxwEQowI6CAguELEDEIMBOggIABCABBCxA1CmGFjcIGD_IWgBcAB4AoABlAGIAbwEkgEDOC4ymAEAoAEBsAEA&sclient=gws-wiz"
driver = webdriver.Chrome(service=Service("C:/chrome/chromedriver.exe"))
driver.get(url)

In [108]:
wait = WebDriverWait(driver, 5)

def scrape_industry(company):
    
    time.sleep(1)
    
    try:
        # Enter in company
        driver.find_element(By.CSS_SELECTOR, '#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef.M2vV3').click()
        search = driver.find_element(By.NAME, "q")
        search.send_keys(company)
        driver.find_element(By.CLASS_NAME,'Tg7LZd').click()
    
    except: 
        pass
    
    try:
        # find industry
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'YhemCb')))
        industry = driver.find_element(By.CLASS_NAME, 'YhemCb').text

        return [industry]
    except: 
        pass
    
    try: 
        wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div/div[10]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/div/div/div[2]/div')))
        industry = driver.find_element(By.XPATH, '/html/body/div[7]/div/div[10]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/div/div/div[2]/div').text

        return [industry]
    
    except:
        print(company)
        return [None]

In [22]:
count = 0

for i, row in scrape_df.iterrows():
    company = row['company_name']
    industry = scrape_industry(company)
    row['industry'] = industry
    
    if count % 50 == 1:
        time.sleep(3)
    
    count += 1

1exchange
21 United Holdings Pte. Ltd.
360f (singapore) Pte. Ltd.
3star Consulting Pte. Ltd.
5 Stones Hrm Pte. Ltd.
99 Group
A1g0 Pte. Ltd.
ALPHALAB CAPITAL PTE. LTD.
Abeam Consulting (singapore) Pte. Ltd.
Aber Care Pte. Ltd.
Aboitiz Data Innovation Pte. Ltd.
Accedo.tv
Accenture Sg Services Pte. Ltd.
Acronis Asia Research And Development Pte. Ltd.
Ad Astra Consultants Private Ltd.
Advance Recruitment Pte. Ltd.
Affinidi
Airbus pioneers
Airtrunk Singapore Pte. Ltd.
Alibaba Cloud (singapore) Private Limited
Allegis Global Solutions (singapore) Pte. Ltd.
Allied Outsourcing Services Pte. Ltd.
Alpha Impact
Alphalab Capital Pte. Ltd.
Alumni Services Pte. Ltd.
Alvisual Pte. Ltd.
Ampstek
Anchor Global Consulting Pte. Ltd.
Anotech Energy Singapore Pte. Ltd.
Antaes Asia Pte. Ltd.
Apex Employment Agency (pte.) Ltd.
Applied Value Technologies Pte. Ltd.
Aquient Pte. Ltd.
Ariston Services Pte. Ltd.
Artefact
Asia Innovatory Management School
Asia Wealth Platform Pte. Ltd.
Asian Fintech
Aspial Corporat

Mongodb Singapore Pte. Ltd.
Moody's Analytics Singapore Pte. Ltd.
Motional Singapore Pte. Limited
Mqjc Technology Pte. Ltd.
Mr Singapore Pte. Ltd.
Msights Asia Pacific Pte. Ltd.
NTUC Enterprise Co-operative Limited
NTUC LearningHub
Nalagenetics Pte. Ltd.
Natsoft Asia Pte. Ltd.
Ncs Communications Engineering Pte. Ltd.
Nec Corporation India Private Limited Singapore Branch
Neo4j
Neo4j Singapore Pte. Ltd.
Net6tem Pte. Ltd.
Network Guard
Network Guard Pte. Ltd.
Neuroncredit Pte. Ltd.
New Bit Technology Pte. Ltd.
New Relic Singapore Pte. Ltd.
Nexmo Pte. Ltd.
Nice Systems (singapore) Pte. Ltd.
Ninja Logistics sdn bhd
Nortonlifelock Singapore Pte. Ltd.
Ntt Singapore Pte. Ltd.
Ntuc Income Insurance Co-operative Ltd
Nyan Heroes
Okbl Pte. Ltd.
Oldendorff Carriers (singapore) Pte. Ltd.
One North Consulting Pte. Ltd.
Onyx Sg Pte. Ltd.
Open Fabric
Opsis Pte. Ltd.
Oracle Global Services Singapore Pte. Ltd.
Orange Business Services Singapore Pte. Ltd.
Outsystems Singapore Pte. Ltd.
Ox Consultancy Pte

In [32]:
scrape_df

,company_name,industry
0,Hitachi Vantara,[Database manage...
1,Novade Solutions,[Software compan...
2,Partners Group,[Financial insti...
3,1exchange,[None]
4,21 United Holdin...,[None]
...,...,...
2228,i-Sprint Innovat...,[Construction co...
2229,iVS – Intelligen...,[Software compan...
2230,iZeeM,[Business to bus...
2231,iZeno Pte Ltd,[None]


## Merge result to original dataframe

In [33]:
final_df = pd.concat([df_combine[df_combine['industry'].isnull() == False], scrape_df])
final_df

,company_name,industry
22,AAX,[Financial Servi...
23,ABB Limited,[Electronics Man...
26,AECOM,[Construction & ...
29,AIA Group,[Insurance]
32,AMD,"[Semiconductor, ..."
...,...,...
2228,i-Sprint Innovat...,[Construction co...
2229,iVS – Intelligen...,[Software compan...
2230,iZeeM,[Business to bus...
2231,iZeno Pte Ltd,[None]


In [94]:
for i, row in final_df.iterrows():
    company = row['company_name']
    df['industry'].loc[df['company_name'] == company] = str(row['industry'])

In [99]:
df

,Unnamed: 0,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack
0,0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set()
1,1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set()
2,2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '..."
3,3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...
4,4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,2896,DevOps Engineer,2C2P,Permanent,"['GitLab', 'HTTP...",2C2P is looking ...,['Software compa...,2022-01-11,https://www.node...,nodeflair,7000.0,10000.0,NaN,3.0,is looking for ...,"{'amazon web', '...","{' GitLabCI', ' ..."
11444,2897,DevOps Engineer,FINXFLO,Permanent,"['Strategy', 'Sh...",Alpha Stone Capi...,['Bank in Singap...,2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,NaN,alpha stone capi...,set(),"{'Strategy', ' S..."
11445,2898,DevOps Engineer,Quilt.AI,Permanent,"['Next.js', 'Doc...",As part of a gro...,[None],2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,3.0,as part of a gro...,set(),"{' Cloudflare', ..."
11446,2899,Software Engineer,Zoku Integrated ...,Permanent,"['API', 'Magento...",We are hiring so...,[None],2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,3.0,we are hiring so...,set(),"{'API', ' Magent..."


In [100]:
# convert to csv
df.to_csv('../data/cleaned/Posts_Industry.csv', index=False)

# Cleaning

In [69]:
df = pd.read_csv('../data/cleaned/Posts_Industry.csv')
df

,Unnamed: 0,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack
0,0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set()
1,1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set()
2,2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '..."
3,3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...
4,4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11443,2896,DevOps Engineer,2C2P,Permanent,"['GitLab', 'HTTP...",2C2P is looking ...,['Software compa...,2022-01-11,https://www.node...,nodeflair,7000.0,10000.0,NaN,3.0,is looking for ...,"{'amazon web', '...","{' GitLabCI', ' ..."
11444,2897,DevOps Engineer,FINXFLO,Permanent,"['Strategy', 'Sh...",Alpha Stone Capi...,['Bank in Singap...,2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,NaN,alpha stone capi...,set(),"{'Strategy', ' S..."
11445,2898,DevOps Engineer,Quilt.AI,Permanent,"['Next.js', 'Doc...",As part of a gro...,[None],2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,3.0,as part of a gro...,set(),"{' Cloudflare', ..."
11446,2899,Software Engineer,Zoku Integrated ...,Permanent,"['API', 'Magento...",We are hiring so...,[None],2022-01-11,https://www.node...,nodeflair,NaN,NaN,NaN,3.0,we are hiring so...,set(),"{'API', ' Magent..."


In [38]:
def str_list1(text):
    if text == 'NaN':
        return np.nan
    
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace("'", '')
    mid_working = text.strip().split(',')
    to_be_returned = mid_working[0]
    return to_be_returned

def str_list2(text):
        if text == 'NaN':
        return np.nan
    
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace("'", '')
    
    return text

In [39]:
df['mid'] = df['industry'].apply(str_list)

In [40]:
df.head()

,Unnamed: 0,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack,mid
0,0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set(),Banking/Accounti...
1,1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set(),Internet/E-commerce
2,2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '...",Corporate office
3,3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...,Semi conductor s...
4,4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...,Semi conductor s...


In [41]:
df['mid'].unique()

array(['Banking/Accounting/Financial Services', 'Internet/E-commerce',
       'Corporate office', 'Semi conductor supplier in Singapore',
       'Employment agency in Singapore', 'Software company in Singapore',
       'Recruiter in Singapore', 'BPO company in Singapore', 'None',
       'Bank in Singapore', 'Telecommunications company',
       'Information Technology & Services',
       'Computer security service in Singapore',
       'Consulting/Advisory Services', 'Computer consultant in Singapore',
       'Contractor in Singapore', 'Association or organization',
       'Training provider in Singapore',
       'Employment consultant in Singapore', 'Consultant in Singapore',
       'Telecommunications service provider in Singapore',
       'IT/Computers - Software', 'Software company in London',
       'Business to business service in Singapore',
       'Employment search service in Singapore',
       'Construction company in Singapore', 'Recruitment/Staffing/RPO',
       'Marketing a

In [62]:
def cleaning(text): 
    text = text.replace('in Singapore', '')
    text = text.replace("in London", "")
    text = text.replace("in Pune", "")
    text = text.replace("in South Jakarta", "")
    text = text.replace("in Hong Kong", "")
#     text = text.replace("Bank", "Banking/Accounting/Financial Services")
#     text = text[0:-1]
    
    return text

In [63]:
df['mid'] = df['industry'].apply(cleaning)

In [64]:
df['mid'].value_counts()

[None]                                                                               1614
['Software company ']                                                                 711
['Internet/E-commerce']                                                               649
['IT/Computers - Software']                                                           446
['Corporate office']                                                                  421
                                                                                     ... 
['Logistics & Supply Chain']                                                            1
['Banking/Accounting/Financial Services', ' IT/Computers - Software']                   1
['Club ']                                                                               1
['Mining company']                                                                      1
['ITES/BPO', ' IT/Computers - Hardware & Networking', ' IT/Computers - Software']       1
Name: mid,

In [65]:
df['mid'] = df['mid'].apply(str_list)

In [66]:
df['mid'].unique()

array(['Banking/Accounting/Financial Services', 'Internet/E-commerce',
       'Corporate office', 'Semi conductor supplier', 'Employment agency',
       'Software company', 'Recruiter', 'BPO company', 'None', 'Bank',
       'Telecommunications company', 'Information Technology & Services',
       'Computer security service', 'Consulting/Advisory Services',
       'Computer consultant', 'Contractor', 'Association or organization',
       'Training provider', 'Employment consultant', 'Consultant',
       'Telecommunications service provider', 'IT/Computers - Software',
       'Software company ', 'Business to business service',
       'Employment search service', 'Construction company',
       'Recruitment/Staffing/RPO', 'Marketing agency',
       'Automation company', 'Commercial cleaning service',
       'Biotechnology company',
       'IT/Software Development - Network Administration/Security',
       'Internet service provider', 'Computer support and services',
       'Security syste

In [67]:
df['mid'].value_counts()

None                                     1614
IT/Computers - Software                  1383
Software company                          730
Internet/E-commerce                       724
Banking/Accounting/Financial Services     552
                                         ... 
Talent agency                               1
Air conditioning system supplier            1
Biotechnology                               1
Toy company                                 1
Corporation                                 1
Name: mid, Length: 284, dtype: int64

In [32]:
df.to_csv('../data/cleaned/Posts_Industry2.csv')

# OKAY!

In [70]:
def str_list1(text):
    if text == 'NaN':
        return np.nan
    
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace("'", '')
    mid_working = text.strip().split(',')
    to_be_returned = mid_working[0]
    return to_be_returned

def str_list2(text):
    if text == 'NaN':
        return np.nan
    
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace("'", '')
    
    return text



def cleaning(text): 
    text = text.replace('in Singapore', '')
    text = text.replace("in London", "")
    text = text.replace("in Pune", "")
    text = text.replace("in South Jakarta", "")
    text = text.replace("in Hong Kong", "")
#     text = text.replace("Bank", "Banking/Accounting/Financial Services")
#     text = text[0:-1]
    
    return text

df['cleaned_industry'] = df['industry'].apply(str_list)
df['cleaned_industry'] = df['cleaned_industry'].apply(cleaning)
df['cleaned_industry'] = df['cleaned_industry'].apply(str_list)
df['cleaned_industry'].value_counts()

None                                            1614
IT/Computers - Software                         1383
Software company                                 784
Internet/E-commerce                              724
Banking/Accounting/Financial Services            552
                                                ... 
Air conditioning system supplier                   1
Biotechnology                                      1
Toy company                                        1
Computers and information technology company       1
Corporation                                        1
Name: cleaned_industry, Length: 282, dtype: int64

In [71]:
df.head()

,Unnamed: 0,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack,cleaned_industry
0,0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set(),Banking/Accounti...
1,1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set(),Internet/E-commerce
2,2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '...",Corporate office
3,3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...,Semi conductor s...
4,4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...,Semi conductor s...


In [76]:
df.drop(columns = "Unnamed: 0", inplace = True)

In [77]:
df.head()

,job_title,company_name,job_type,tech_stack,job_desc,industry,date_posted,url,source,lower_salary,upper_salary,lower_year,upper_year,cleaned_job_desc,extracted_tech_stack,all_tech_stack,cleaned_industry
0,Senior Python De...,Citi,permanent,NaN,Job Description ...,['Banking/Accoun...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,{'jasminkarma ju...,set(),Banking/Accounti...
1,Senior Partner S...,Amazon,permanent,NaN,Job Description ...,['Internet/E-com...,2022-02-18,https://www.mons...,monster,NaN,NaN,NaN,NaN,job description ...,"{'microsoft', 'a...",set(),Internet/E-commerce
2,IT Technician,Ascend Com Pte. ...,permanent,"['Switches', 'Ma...",Responsibilities...,['Corporate offi...,2022-02-19,https://www.mons...,monster,24000.0,36000.0,2.0,5.0,responsibilities...,{'lan wan'},"{' Windows 7', '...",Corporate office
3,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Key Responsibili...,['Semi conductor...,2022-02-19,https://www.mons...,monster,42000.0,84000.0,2.0,5.0,key responsibili...,{'daytoday'},{' Problem Solvi...,Semi conductor s...
4,Customer Engineer,Applied Material...,permanent,"['Scalability', ...",Has developed sp...,['Semi conductor...,2022-02-19,https://www.mons...,monster,46800.0,93600.0,2.0,5.0,has developed sp...,set(),{' Problem Solvi...,Semi conductor s...


In [79]:
df.to_csv('../data/cleaned/Posts_Industry2.csv')